In [1]:
!pip install pytorch_pretrained_bert
!pip install pdfminer.six

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 5.6MB 2.7MB/s 
     |████████████████████████████████| 13.7MB 236kB/s 


In [3]:
from torch.utils.data import TensorDataset, DataLoader
from keras.preprocessing.sequence import pad_sequences
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfparser import PDFSyntaxError
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal
from spacy.matcher import Matcher

from datetime import datetime
from dateutil import relativedelta
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import math
import numpy as np
import torch
import os
import json
import re
import torch
import nltk
import spacy
import string
import io
import multiprocessing as mp

nltk.download('stopwords')
nltk.download('wordnet')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
class ResumeParser(object):

    def __init__(
        self,
        resume,
        skills_file=None,
        custom_regex=None
    ):
        super(ResumeParser, self).__init__()
        nlp = spacy.load('en_core_web_sm')
        custom_nlp = spacy.load('pyre_model')

        self.__skills_file = skills_file
        self.__custom_regex = custom_regex
        self.__matcher = Matcher(nlp.vocab)
        self.__details = {
            'name': [],
            'designation': [],
            'degree': [],
            'projects':[],
            'address':[],
            'college': None,
            'graduation':[],
            'email': None,
            'location':[],
            'links':None,
            'mobile_number': None,
            'companies': None,
            'skills': None,
            'achievements':[],
            'certifications':[],
            'experience': None,
            'no_of_pages': None,
            'total_experience': None,
        }
        self.NAME_PATTERN = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
        self.EDUCATION = [
             'A.A.','Associate of Arts','A.A.S.','Associate of Applied Science'
             'A.A.T.','Associate in Applied Tech','A.O.T.','Associate in Occupational Technology'
             'A.S.','Associate of Science','B.A.','Bachelor of Arts',
             'B.A.B.A.','Bach of Arts of Business Administration','B.A.Com','B.A.E.'
             'Bachelor of Arts in Education, Bachelor of Art Education, Bachelor of Aerospace Engineering'
             ,'B.Ag','Bachelor of Agriculture','B.Arch','Bachelor of Architecture','B.B.A'
             ,'Bachelor of Arts in Communication','Bachelor of Business Administration','B.C.E.','Bachelor of Civil Engineering'
             'B.Ch.E','Bachelor of Chemical Engineering','B.D.','Bachelor of Divinity','B.E.','Bachelor of Education, Bachelor of Engineering',
             'B.E.E.','Bachelor of Electrical Engineering','B.F.A.','Bachelor of Fine Arts',
             'Bachelor','Master','Doctor'
            'BE', 'B.E.', 'B.E', 'BS', 'B.S', 'ME', 'M.E',
            'M.E.', 'MS', 'M.S', 'BTECH', 'MTECH',
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
        ]
        self.NOT_ALPHA_NUMERIC = r'[^a-zA-Z\d]'
        self.NUMBER = r'\d+'
        self.MONTHS_SHORT = r'''(Jan)|(Feb)|(Mar)|(Apr)|(May)|(Jun)|(Jul)
                   |(Aug)|(Sep)|(Oct)|(Nov)|(Dec)'''
        self.MONTHS_LONG = r'''(january)|(february)|(march)|(april)|(may)|(june)|(july)|
                   (august)|(september)|(october)|(november)|(december)'''
        self.MONTH = r'(' + self.MONTHS_SHORT + r'|' + self.MONTHS_LONG + r')'
        self.YEAR = r'(((20|19)(\d{2})))'
        self.STOPWORDS = set(stopwords.words('english'))
        self.RESUME_SECTIONS_PROFESSIONAL = [
                    'experience','education','interests','professional experience','publications',
                    'skills', 'certifications','objective','career objective',
                    'summary','leadership'
                ]
        self.RESUME_SECTIONS_GRAD = [
                    'accomplishments','experience','education', 'interests','projects','professional experience',
                    'publications','skills','certifications',  'objective','career objective','summary','leadership'
                ]

        self.__resume = resume
        if not isinstance(self.__resume, io.BytesIO):
            ext = os.path.splitext(self.__resume)[1].split('.')[1]
        else:
            ext = self.__resume.name.split('.')[1]
        self.__text_raw = self.extract_text(self.__resume, '.' + ext)
        self.textbert=self.extract_text(self.__resume, '.' + ext)

        self.__text = ' '.join(self.__text_raw.split())
        self.__nlp = nlp(self.__text)
        self.__custom_nlp = custom_nlp(self.__text_raw)
        self.__noun_chunks = list(self.__nlp.noun_chunks)
        self.__get_basic_details()
        self.ret={}
        self.uni=[]
        self.tag2idx={'B-Achievements': 1,'B-Address': 28,'B-CRT': 65,'B-Certifications': 55,'B-College': 41,'B-Companies': 74,
                      'B-Degree': 27,'B-Designation': 0,'B-Email': 43, 'B-Graduation': 33, 'B-Links': 8, 'B-Location': 42,
                      'B-Name': 50,'B-Skills': 51,'B-UNKNOWN': 19,'B-University': 25, 'B-YOE': 36, 'B-projects': 53,
                      'B-state': 70, 'B-training': 6,'I-Achievements': 40,'I-Address': 66, 'I-CRT': 3, 'I-Certifications': 60,
                      'I-College': 14, 'I-Companies': 54, 'I-Degree': 67, 'I-Designation': 62,'I-Email': 38,'I-Graduation': 15,'I-Links': 72,
                      'I-Location': 61,'I-Name': 59,'I-Skills': 4,'I-UNKNOWN': 9,'I-University': 77,'I-YOE': 13,'I-projects': 37,'I-training': 63,
                      'L-Achievements': 31,'L-Address': 22,'L-CRT': 73,'L-Certifications': 64,'L-College': 78,'L-Companies': 46,'L-Degree': 16,
                      'L-Designation': 12,'L-Email': 2,'L-Graduation': 21,'L-Links': 11,'L-Location': 35, 'L-Name': 47, 'L-Skills': 71,
                      'L-UNKNOWN': 56,'L-University': 57, 'L-YOE': 23, 'L-projects': 68, 'L-state': 52, 'L-training': 49,'O': 48,
                      'U-Address': 17, 'U-CRT': 69, 'U-Certifications': 34, 'U-College': 18, 'U-Companies': 76, 'U-Degree': 44, 'U-Designation': 39,
                      'U-Email': 58, 'U-Graduation': 79, 'U-Links': 26, 'U-Location': 20, 'U-Name': 30,
                     'U-Skills': 29,'U-UNKNOWN': 7,'U-YOE': 75, 'U-links': 5, 'U-state': 10, 'X': 24, '[CLS]': 32, '[SEP]': 45}
 
        self.idx2tag = {self.tag2idx[key] : key for key in self.tag2idx.keys()}

        self.res_idx_tag={'B-Achievements': 'achievements', 'B-Address': 'address','B-CRT': 'CRT','B-Certifications': 'certifications',
                'B-College': 'college', 'B-Companies': 'companies','B-Degree': 'degree', 'B-Designation': 'designation','B-Email': 'email',
  '              B-Graduation': 'graduation','B-Links': 'links','B-Location': 'location', 'B-Name': 'name','B-Skills': 'skills',
                'B-UNKNOWN': 'UNKNOWN','B-University': 'university', 'B-YOE': 'YOE', 'B-projects': 'projects', 
                'B-state': 'state', 'B-training': 'training', 'I-Achievements': 'achievements',
                'I-Address': 'address', 'I-CRT': 'CRT', 'I-Certifications': 'certifications', 
                'I-College': 'college','I-Companies': 'companies', 'I-Degree': 'degree', 'I-Designation': 'designation',
                'I-Email': 'email', 'I-Graduation': 'graduation', 'I-Links': 'links', 'I-Location': 'location','I-Name': 'name',
                'I-Skills': 'skills','I-UNKNOWN': 'UNKNOWN', 'I-University': 'university','I-YOE': 'year of experince','I-projects': 'projects',
                'I-training': 'training', 'L-Achievements': 'achievements', 'L-Address': 'address',
                'L-CRT': 'CRT', 'L-Certifications': 'certifications', 'L-College': 'college', 'L-Companies':'companies', 
                'L-Degree': 'degree','L-Designation': 'designation','L-Email': 'email','L-Graduation': 'graduation','L-Links': 'links',
                'L-Location': 'location','L-Name':'name','L-Skills': 'skills', 'L-UNKNOWN': 'UNKNOWN',
                'L-University': 'university','L-YOE': 'year of experince','L-projects': 'projects','L-state': 'state',
                'L-training': 'training', 'O': 'O','U-Address': 'address','U-CRT': 'CRT','U-Certifications': 'certifications',
                'U-College': 'college','U-Companies': 'companies','U-Degree': 'degree','U-Designation': 'designation',
                'U-Email': 'email','U-Graduation': 'graduation','U-Links': 'links','U-Location': 'location','U-Name': 'name',
                'U-Skills': 'skills','U-UNKNOWN': 'UNKNOWN','U-YOE': 'year of experince','U-links': 'links','U-state': 'state',
                 'X': 'X', '[CLS]': '[CLS]','[SEP]': '[SEP]'}
          
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(self.tag2idx))
        self.model.load_state_dict(torch.load('ner_resume_2.bin',map_location=torch.device(self.device)))
        self.model.to(self.device)





    def Resume(self):

      retbert=self.predict()
      retspacy=self.get_extracted_data()

      if 'achievements' in retbert:
        retspacy['achievements']=retbert['achievements']
      if 'name' in retbert:
        retspacy['name']=retbert['name']
      if 'college' in retbert:
        retspacy['college']=retbert['college']
      if 'location' in retbert:
        retspacy['location']=retbert['location']
      if 'companies' in retbert:
        retspacy['companies']=retbert['companies']
      if 'degree' in retbert:
        retspacy['degree']=retbert['degree']
      if 'designation' in retbert:
        retspacy['designation']=retbert['designation']

      if 'graduation' in retbert:
        retspacy['graduation']=retbert['graduation']
      
      if 'address' in retbert:
        retspacy['address']=retbert['address']

      if retspacy['skills'] == [] or retspacy['skills'] ==None :
        if 'skills' in retbert:
          retspacy['skills']=retbert['skills'] 
  


      return retspacy

    def predict(self):
      sent=self.textbert.splitlines()
      sent=[st for st in sent if st !='' or st not in string.punctuation]

      words = [self.tokenizer.tokenize(snt) for snt in sent]
      sent = ["[CLS] " + sentence + " [SEP]" for sentence in sent]
      text_sent=[self.tokenizer.tokenize(snt) for snt in sent]
      input_ids = pad_sequences([self.tokenizer.convert_tokens_to_ids(txt) for txt in text_sent],
                          maxlen=512, dtype="long", truncating="post", padding="post")


      attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

      test_data = TensorDataset(torch.tensor(input_ids), torch.tensor(attention_masks))
      test_dataloader = DataLoader(test_data, batch_size=1)


      res_tag=[]

      self.model.eval()
      ind=0
      with torch.no_grad():
        for batch in test_dataloader:
            batch = tuple(t.to(self.device) for t in batch)
            input_ids, input_mask = batch
        
            logits = self.model(input_ids, token_type_ids=None, attention_mask=input_mask)
            logits = logits.detach().cpu().numpy()
            logits = [list(p) for p in np.argmax(logits, axis=2)]
            input_mask = input_mask.to('cpu').numpy()
    
            for i,mask in enumerate(input_mask):
                temp_1 = [] # Real one
                for j, m in enumerate(mask):
                    if m:
                        if  self.idx2tag[logits[i][j]] != "[CLS]" and self.idx2tag[logits[i][j]] != "[SEP]" : 
                            temp_1.append(self.res_idx_tag[self.idx2tag[logits[i][j]]])
                    else:
                        break
                res_tag.append(temp_1)

        
            self.process([res_tag[ind]],words[ind])
            ind+=1
      
      return self.ret
    def process(self,res_tag,words):
      tmp,indx,ents='',0,[]
      while (indx < len(res_tag[0])):
         if res_tag[0][indx]!='X':
          tmp+=words[indx]
          ents.append(res_tag[0][indx])
          indx+=1
          while(indx<len(res_tag[0])):
              if res_tag[0][indx]=='X':
                  tmp+=words[indx][2:]
                  indx+=1
              else:
                  break
          tmp+=' '
      res={}

      for ent,tm in zip(ents,tmp.split(' ')):
        if ent in res:
            tmpt=res[ent]
            tmpt+=' '+tm
            res[ent]=tmpt
        else:
            res[ent]=tm
        
      for key, value in res.items() :
        if key !="O":
          if key in self.ret :
            if key !='name' and key !='location':
              tmp=self.ret[key]
              ok=1
              for k,v in tmp[0].items():
                 if str(value).lower()==v or str(value).lower()  in self.uni:
                   ok=0
              if ok:
                tmp.append({'name':value})
                self.uni.append(str(value).lower())
                self.ret[key]=tmp
          else:
            self.ret[key]=[{'name':value}]
            self.uni.append(str(value).lower())

   

    def extract_text_from_pdf(self,pdf_path):
      if not isinstance(pdf_path, io.BytesIO):
          with open(pdf_path, 'rb') as fh:
            try:
                for page in PDFPage.get_pages(
                                fh,
                                caching=True,
                                check_extractable=True
                ):
                    resource_manager = PDFResourceManager()
                    fake_file_handle = io.StringIO()
                    converter = TextConverter(
                        resource_manager,
                        fake_file_handle,
                        codec='utf-8',
                        laparams=LAParams()
                    )
                    page_interpreter = PDFPageInterpreter(
                        resource_manager,
                        converter
                    )
                    page_interpreter.process_page(page)

                    text = fake_file_handle.getvalue()
                    yield text

                    converter.close()
                    fake_file_handle.close()
            except PDFSyntaxError:
              return
      else:
        try:
            for page in PDFPage.get_pages(
                pdf_path,
                caching=True,
                check_extractable=True
            ):
                resource_manager = PDFResourceManager()
                fake_file_handle = io.StringIO()
                converter = TextConverter(
                    resource_manager,
                    fake_file_handle,
                    codec='utf-8',
                    laparams=LAParams()
                )
                page_interpreter = PDFPageInterpreter(
                    resource_manager,
                    converter
                )
                page_interpreter.process_page(page)

                text = fake_file_handle.getvalue()
                yield text

                converter.close()
                fake_file_handle.close()
        except PDFSyntaxError:
            return
    def get_number_of_pages(self,file_name):
      try:
        if isinstance(file_name, io.BytesIO):
            count = 0
            for page in PDFPage.get_pages(
                        file_name,
                        caching=True,
                        check_extractable=True
            ):
                count += 1
            return count
        else:
            if file_name.endswith('.pdf'):
                count = 0
                with open(file_name, 'rb') as fh:
                    for page in PDFPage.get_pages(
                            fh,
                            caching=True,
                            check_extractable=True
                    ):
                        count += 1
                return count
            else:
                return None
      except PDFSyntaxError:
        return None

    def extract_entity_sections_grad(self,text):
    
      text_split = [i.strip() for i in text.split('\n')]
      entities = {}
      key = False
      for phrase in text_split:
        if len(phrase) == 1:
            p_key = phrase
        else:
            p_key = set(phrase.lower().split()) & set(self.RESUME_SECTIONS_GRAD)
        try:
            p_key = list(p_key)[0]
        except IndexError:
            pass
        if p_key in self.RESUME_SECTIONS_GRAD:
            entities[p_key] = []
            key = p_key
        elif key and phrase.strip():
            entities[key].append(phrase)
      return entities
    def extract_entities_wih_custom_model(self,custom_nlp_text):
    
      entities = {}
      for ent in custom_nlp_text.ents:
        if ent.label_ not in entities.keys():
            entities[ent.label_] = [ent.text]
        else:
            entities[ent.label_].append(ent.text)
      for key in entities.keys():
        entities[key] = list(set(entities[key]))
      return entities

    def get_total_experience(self,experience_list):
    
      exp_ = []
      for line in experience_list:
        experience = re.search(
            r'(?P<fmonth>\w+.\d+)\s*(\D|to)\s*(?P<smonth>\w+.\d+|present)',
            line,
            re.I
        )
        if experience:
            exp_.append(experience.groups())
      total_exp = sum(
        [self.get_number_of_months_from_dates(i[0], i[2]) for i in exp_]
      )
      total_experience_in_months = total_exp
      return total_experience_in_months

    def get_number_of_months_from_dates(self,date1, date2):
    
      if date2.lower() == 'present':
        date2 = datetime.now().strftime('%b %Y')
      try:
        if len(date1.split()[0]) > 3:
            date1 = date1.split()
            date1 = date1[0][:3] + ' ' + date1[1]
        if len(date2.split()[0]) > 3:
            date2 = date2.split()
            date2 = date2[0][:3] + ' ' + date2[1]
      except IndexError:
        return 0
      try:
        date1 = datetime.strptime(str(date1), '%b %Y')
        date2 = datetime.strptime(str(date2), '%b %Y')
        months_of_experience = relativedelta.relativedelta(date2, date1)
        months_of_experience = (months_of_experience.years
                                * 12 + months_of_experience.months)
      except ValueError:
        return 0
      return months_of_experience

    def extract_entity_sections_professional(self,text):
  
      text_split = [i.strip() for i in text.split('\n')]
      entities = {}
      key = False
      for phrase in text_split:
        if len(phrase) == 1:
            p_key = phrase
        else:
            p_key = set(phrase.lower().split()) \
                    & set(self.RESUME_SECTIONS_PROFESSIONAL)
        try:
            p_key = list(p_key)[0]
        except IndexError:
            pass
        if p_key in self.RESUME_SECTIONS_PROFESSIONAL:
            entities[p_key] = []
            key = p_key
        elif key and phrase.strip():
            entities[key].append(phrase)
      return entities

    def extract_email(self,text):
      email = re.findall(r"([^@|\s]+@[^@]+\.[^@|\s]+)", text)
      if email:
          try:
            return email[0].split()[0].strip(';')
          except IndexError:
            return None

    def extract_name(self,nlp_text, matcher):
   
      pattern = [self.NAME_PATTERN]

      matcher.add('NAME', None, *pattern)

      matches = matcher(nlp_text)

      for _, start, end in matches:
          span = nlp_text[start:end]
          if 'name' not in span.text.lower():
            return span.text

    def extract_mobile_number(self,text, custom_regex=None):
    
      if not custom_regex:
          mob_num_regex = r'''(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})'''
          phone = re.findall(re.compile(mob_num_regex), text)
      else:
          phone = re.findall(re.compile(custom_regex), text)
      
      
      if phone:
        number = [{'name':phonei} for phonei in phone if len(phonei)>9]

        return number


    def extract_links(self,text, custom_regex=None):
    
      if not custom_regex:
          links_regex = r'''(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)'''
          links = re.findall(re.compile(links_regex), text)
      else:
          links = re.findall(re.compile(links_regex), text)
      
      
      if links:
        links=list(set(links))
        links = [{'name':linksi} for linksi in links]

        return links


    def extract_skills(self,nlp_text, noun_chunks, skills_file=None):
   
      tokens = [token.text for token in nlp_text if not token.is_stop]
      if not skills_file:
        data = pd.read_csv('/content/drive/My Drive/skills.csv')
      else:
        data = pd.read_csv(skills_file)
      skills = list(data.columns.values)
      skillset = []
      for token in tokens:
          if token.lower() in skills:
              skillset.append(token)

      for token in noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
      skillset= [i.capitalize() for i in set([i.lower() for i in skillset])]
      return [{'name':token} for token in skillset]


    def cleanup(self,token, lower=True):
      if lower:
        token = token.lower()
      return token.strip()

    def extract_education(self,nlp_text):
    
      edu = {}
      try:
        for index, text in enumerate(nlp_text):
            for tex in text.split():
                tex = re.sub(r'[?|$|.|!|,]', r'', tex)
                if tex.upper() in EDUCATION and tex not in STOPWORDS:
                    edu[tex] = text + nlp_text[index + 1]
      except IndexError:
        pass

      education = []
      for key in edu.keys():
          year = re.search(re.compile(YEAR), edu[key])
          if year:
            education.append((key, ''.join(year.group(0))))
          else:
            education.append(key)

      if education !=[]:
        education=[{'education':educationi} for educationi in education]
      return education


    def extract_experience(self,resume_text):
   
      wordnet_lemmatizer = WordNetLemmatizer()
      stop_words = set(stopwords.words('english'))

      word_tokens = nltk.word_tokenize(resume_text)

      filtered_sentence = [
            w for w in word_tokens if w not
            in stop_words and wordnet_lemmatizer.lemmatize(w)
            not in stop_words
        ]
      sent = nltk.pos_tag(filtered_sentence)

      cp = nltk.RegexpParser('P: {<NNP>+}')
      cs = cp.parse(sent)

  

      test = []

      for vp in list(
        subtrees(filter=lambda x: x.label() == 'P')
      ):
        test.append(" ".join([
            i[0] for i in vp.leaves()
            if len(vp.leaves()) >= 2])
        )

      x = [
        x[x.lower().index('experience') + 10:]
        for i, x in enumerate(test)
        if x and 'experience' in x.lower()
      ]
      if x !=[]:
        x=[{'experince':xs} for xs in x]
      return x

    def extract_text(self,file_path, extension):
      text = ''
      if extension == '.pdf':
        for page in self.extract_text_from_pdf(file_path):
            text += ' ' + page
      return text

    
    
    
    
    def get_extracted_data(self):
        return self.__details

    def __get_basic_details(self):
        cust_ent = self.extract_entities_wih_custom_model(
                            self.__custom_nlp
                        )
        name = self.extract_name(self.__nlp, matcher=self.__matcher)
        email = self.extract_email(self.__text)
        mobile = self.extract_mobile_number(self.__text, self.__custom_regex)
        skills = self.extract_skills(
                    self.__nlp,
                    self.__noun_chunks,
                    self.__skills_file
                )
        links=self.extract_links(self.__text)
       
        entities = self.extract_entity_sections_grad(self.__text_raw)

        try:
            self.__details['name'] = cust_ent['Name'][0]
        except (IndexError, KeyError):
            self.__details['name'] = name

        self.__details['email'] = email

        self.__details['mobile_number'] = mobile

        self.__details['skills'] = skills

        self.__details['links'] = links

        try:
            self.__details['college'] = entities['College']
        except KeyError:
            pass

        try:
            self.__details['degree'] = cust_ent['Degree']
        except KeyError:
            pass

        try:
            self.__details['designation'] = cust_ent['Designation']
        except KeyError:
            pass

        try:
            self.__details['companies'] = cust_ent['Companies worked at']
        except KeyError:
            pass

        try:
            self.__details['experience'] = entities['experience']
            try:
                exp = round(
                    self.get_total_experience(entities['experience']) / 12,
                    2
                )
                self.__details['total_experience'] = exp
            except KeyError:
                self.__details['total_experience'] = 0
        except KeyError:
            self.__details['total_experience'] = 0
        self.__details['no_of_pages'] = self.get_number_of_pages(
                                            self.__resume
                                        )
        return


In [5]:
ResumeParser("Ahmed Muhammad.pdf").Resume()

100%|██████████| 404400730/404400730 [00:35<00:00, 11459242.27B/s]


{'achievements': [{'name': 'in Contests'}],
 'address': [],
 'certifications': [],
 'college': [{'name': 'Assiut University'}, {'name': 'Assiut'}],
 'companies': [{'name': 'iNetworks'}],
 'degree': [],
 'designation': [{'name': 'Computer Science Student'},
  {'name': 'Data Scientist'}],
 'email': 'ahmedsmara33@gmail.com',
 'experience': ['Computers and Information, Assiut University — Bachelor’s degree',
  'Undergraduate (Aug 2016 –Aug 2020)',
  'Intern Data Scientist at Wakeb  — Egypt',
  '- Learned and deployed machine learning models and algorithms.',
  '- Worked on chat-bots and the dialects in Arabic language.',
  '- Implemented sentiment analysis model on Arabic sentences.',
  '- Analysis of the content of social networks into negative and positive impressions.',
  '(Jan 2020– Mar 2020)',
  'Intern Data Scientist at iNetworks — Egypt',
  '- Learned and deployed machine learning models and algorithms.',
  '- Implemented text classification model, recognition system and Sentiment a